In [1]:
import numpy as np
import torch
import sys
sys.path.append('../')
from voting_games.werewolf_env_v0 import plurality_env, Roles, Phase
from notebooks.learning_agents.models import ActorCriticAgent
from notebooks.learning_agents.utils import play_static_game, play_recurrent_game
from notebooks.learning_agents.static_agents import (
    random_plurality_villager, 
    random_coordinated_plurality_villager, 
    random_agent,
    random_plurality_wolf,
    revenge_plurality_wolf,
    coordinated_revenge_plurality_wolf)
import random
import copy
from tqdm import tqdm
from tabulate import tabulate

/root/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Plurality Voting

Blurb on plurality voting goes here...

We want to see how an static agents vs static wolves fare, before training our PPO agents to hopefully learn to do better

## Pre-Training Baselines

To properly asses our agents, we need baselines. For this purpose we have totally random villagers and semi-random villagers that will only vote for agents that are currently alive.

As for wolves, we have the following behaviors:
- wolves that coordinate and each target one villager
- random wolves that do whatever
- revenge wolves that coordinate and target a random villager that targetted a wolf

In [2]:
env = plurality_env(num_agents=10, werewolves=2, num_accusations=2)
observations, _, _, _, _ = env.reset()

obs_size= env.convert_obs(observations['player_0']['observation']).shape[-1]

trained_plurality_agent = ActorCriticAgent({"rec_hidden_size": 128,
                                        "rec_layers": 1, 
                                        "joint_mlp_size": 128,
                                        "split_mlp_size": 128,
                                        "num_votes": 1,
                                        "approval_states": 10},
                                        num_players=10,
                                        obs_size=obs_size)
trained_plurality_agent.load_state_dict(torch.load("../notebooks/stored_agents/lstm_first_no_one_hot_128_128/plurality_agent_10_score_46"))



FileNotFoundError: [Errno 2] No such file or directory: 'stored_agents/lstm_first_no_one_hot_128_128/plurality_agent_10_score_46'

In [2]:
env = plurality_env(num_agents=10, werewolves=2, num_accusations=1)
env.reset()
print(f'10 players, with 2 wolves\n')

revenge_coordinated_wolves = []
revenge_coordinated_wolves.append(play_static_game(env, coordinated_revenge_plurality_wolf, random_plurality_villager, num_times=1000)[0]/1000.0)
revenge_coordinated_wolves.append(play_static_game(env, coordinated_revenge_plurality_wolf, random_agent, num_times=1000)[0]/1000.0)
revenge_coordinated_wolves.append(play_static_game(env, coordinated_revenge_plurality_wolf, random_coordinated_plurality_villager, num_times=1000)[0]/1000.0)

revenge_wolves = []
revenge_wolves.append(play_static_game(env, revenge_plurality_wolf, random_plurality_villager, num_times=1000)[0]/1000.0)
revenge_wolves.append(play_static_game(env, revenge_plurality_wolf, random_agent, num_times=1000)[0]/1000.0)
revenge_wolves.append(play_static_game(env, revenge_plurality_wolf, random_coordinated_plurality_villager, num_times=1000)[0]/1000.0)

coordinated_wolves = []
coordinated_wolves.append(play_static_game(env, random_plurality_wolf, random_plurality_villager, num_times=1000)[0]/1000.0)
coordinated_wolves.append(play_static_game(env, random_plurality_wolf, random_agent, num_times=1000)[0]/1000.0)
coordinated_wolves.append(play_static_game(env, random_plurality_wolf, random_coordinated_plurality_villager, num_times=1000)[0]/1000.0)

random_wolves = []
random_wolves.append(play_static_game(env, random_agent, random_plurality_villager, num_times=1000)[0]/1000.0)
random_wolves.append(play_static_game(env, random_agent, random_agent, num_times=1000)[0]/1000.0)
random_wolves.append(play_static_game(env, random_agent, random_coordinated_plurality_villager, num_times=1000)[0]/1000.0)

print(tabulate([['Coordinated Wolves', *coordinated_wolves], ['Random Wolves', *random_wolves], ['Coorindated revenge wolves', *revenge_coordinated_wolves], ['Revenge wolves', *revenge_wolves]], 
               headers=["Wolf Strategy", "Semi-Smart Villager", "Totally Random Villager", "Coordinated Random Villagers"]))

print("\n")
env = plurality_env(num_agents=15, werewolves=3, num_accusations=1)
env.reset()
print(f'15 players, with 3 wolves\n')

revenge_coordinated_wolves = []
revenge_coordinated_wolves.append(play_static_game(env, coordinated_revenge_plurality_wolf, random_plurality_villager, num_times=1000)[0]/1000.0)
revenge_coordinated_wolves.append(play_static_game(env, coordinated_revenge_plurality_wolf, random_agent, num_times=1000)[0]/1000.0)
revenge_coordinated_wolves.append(play_static_game(env, coordinated_revenge_plurality_wolf, random_coordinated_plurality_villager, num_times=1000)[0]/1000.0)

revenge_wolves = []
revenge_wolves.append(play_static_game(env, revenge_plurality_wolf, random_plurality_villager, num_times=1000)[0]/1000.0)
revenge_wolves.append(play_static_game(env, revenge_plurality_wolf, random_agent, num_times=1000)[0]/1000.0)
revenge_wolves.append(play_static_game(env, revenge_plurality_wolf, random_coordinated_plurality_villager, num_times=1000)[0]/1000.0)

coordinated_wolves = []
coordinated_wolves.append(play_static_game(env, random_plurality_wolf, random_plurality_villager, num_times=1000)[0]/1000.0)
coordinated_wolves.append(play_static_game(env, random_plurality_wolf, random_agent, num_times=1000)[0]/1000.0)
coordinated_wolves.append(play_static_game(env, random_plurality_wolf, random_coordinated_plurality_villager, num_times=1000)[0]/1000.0)

random_wolves = []
random_wolves.append(play_static_game(env, random_agent, random_plurality_villager, num_times=1000)[0]/1000.0)
random_wolves.append(play_static_game(env, random_agent, random_agent, num_times=1000)[0]/1000.0)
random_wolves.append(play_static_game(env, random_agent, random_coordinated_plurality_villager, num_times=1000)[0]/1000.0)




print(tabulate([['Coordinated Wolves', *coordinated_wolves], ['Random Wolves', *random_wolves], ['Coorindated revenge wolves', *revenge_coordinated_wolves], ['Revenge wolves', *revenge_wolves]], 
               headers=["Wolf Strategy", "Semi-Smart Villager", "Totally Random Villager", "Coordinated Random Villagers"]))

10 players, with 2 wolves

Wolf Strategy                 Semi-Smart Villager    Totally Random Villager    Coordinated Random Villagers
--------------------------  ---------------------  -------------------------  ------------------------------
Coordinated Wolves                          0.13                       0.05                            0.31
Random Wolves                               0.653                      0.594                           0.711
Coorindated revenge wolves                  0.246                      0.08                            0.297
Revenge wolves                              0.215                      0.081                           0.329


15 players, with 3 wolves

Wolf Strategy                 Semi-Smart Villager    Totally Random Villager    Coordinated Random Villagers
--------------------------  ---------------------  -------------------------  ------------------------------
Coordinated Wolves                          0.029                      0.